In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('online_retail_data(in).csv')

# 1. Remove duplicates
df.drop_duplicates(inplace=True)

# Replace empty strings with NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# 2. Delete rows with empty StockCode, Invoice, or Quantity
# These are safe to delete as they are useless for any analysis
df.dropna(subset=['StockCode', 'Invoice', 'Quantity'], inplace=True)

# 3. Handle Description and Price
# Create mappings to fill missing values from other rows with the same StockCode
stock_desc_map = df.dropna(subset=['Description']).groupby('StockCode')['Description'].first()
stock_price_map = df.dropna(subset=['Price']).groupby('StockCode')['Price'].first()

df['Description'] = df['Description'].fillna(df['StockCode'].map(stock_desc_map))
df['Price'] = df['Price'].fillna(df['StockCode'].map(stock_price_map))

# Identify adjustment rows: Quantity < 0 AND Price == 0
is_adjustment = (df['Quantity'] < 0) & (df['Price'] == 0)
df['is_adjustment'] = is_adjustment # Create a flag column

# Fill remaining missing descriptions for adjustments
df.loc[is_adjustment & df['Description'].isnull(), 'Description'] = "Unknown Inventory Adjustment"

# Now safely delete rows that are NOT adjustments but still have missing info (truly bad data)
df.dropna(subset=['Description', 'Price'], inplace=True)

# 4. Handle InvoiceDate and Country (Refer to Invoice)
invoice_date_map = df.dropna(subset=['InvoiceDate']).groupby('Invoice')['InvoiceDate'].first()
invoice_country_map = df.dropna(subset=['Country']).groupby('Invoice')['Country'].first()

df['InvoiceDate'] = df['InvoiceDate'].fillna(df['Invoice'].map(invoice_date_map))
df['Country'] = df['Country'].fillna(df['Invoice'].map(invoice_country_map))

# Save
df.to_csv('cleaned_online_retail_data_smart.csv', index=False)

In [ ]:
# 1. Filter for rows where Quantity is less than 0
negative_qty_df = df[df['Quantity'] < 0]

# 2. Filter that result for Invoices that do NOT start with 'C'
# We use the tilde (~) to invert the condition
inventory_adjustments = negative_qty_df[~negative_qty_df['Invoice'].astype(str).str.startswith('C')]

# Display the results
print(f"Found {len(inventory_adjustments)} inventory adjustment rows.")
print(inventory_adjustments.head())
print(inventory_adjustments['Description'].value_counts().head(10))

Found 3070 inventory adjustment rows.
     Invoice StockCode                Description  Quantity      InvoiceDate  \
263   489464     21733               85123a mixed       -96  12/1/2009 10:52   
283   489463     71477                      short      -240  12/1/2009 10:52   
284   489467    85123A                21733 mixed      -192  12/1/2009 10:53   
3114  489655     20683  RAIN GIRL CHILDS UMBRELLA       -44  12/1/2009 17:26   
3162  489660     35956                       lost     -1043  12/1/2009 17:43   

      Price  Customer ID         Country  
263     0.0          NaN  United Kingdom  
283     0.0          NaN  United Kingdom  
284     0.0          NaN  United Kingdom  
3114    0.0          NaN  United Kingdom  
3162    0.0          NaN  United Kingdom  
Description
check                     115
?                          82
damages                    82
damaged                    78
missing                    26
sold as set on dotcom      20
Damaged                    17
t

In [1]:
import pandas as pd
import numpy as np
import csv

# 1. LOAD DATA & CLEAN STOCKCODES
file_path = 'cleaned_online_retail_data_smart.csv'
df = pd.read_csv(file_path)

# STRICT CLEANING: Force Uppercase and Trim Spaces to prevent duplicates
df['StockCode'] = df['StockCode'].astype(str).str.strip().str.upper()

# Sort by date
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df = df.sort_values(by='InvoiceDate')

# 2. CALCULATE INVENTORY IMPACT (BEFORE DELETING ERRORS)
# Define the logic columns
is_return = df['Invoice'].astype(str).str.startswith('C')
is_negative = df['Quantity'] < 0
is_error = (is_negative) & (~is_return) # The rows you want to remove from invoice list but COUNT for stock

def calc_stock_change(row):
    qty = row['Quantity']
    if row['is_error']:
        # Error: Quantity is -10. We want to SUBTRACT 10 from stock.
        # Since qty is -10, adding it (-10) achieves subtraction.
        return qty
    else:
        # Sale: Quantity is 10. We want to SUBTRACT 10. -> returns -10.
        # Return: Quantity is -10. We want to ADD 10. -> returns -(-10) = +10.
        return -qty

df['is_error'] = is_error
df['StockChange'] = df.apply(calc_stock_change, axis=1)

# Sum the net change for every product (using ALL data)
stock_impact_map = df.groupby('StockCode')['StockChange'].sum().to_dict()

# 3. FILTER ERRORS (NOW WE CAN DELETE THEM)
clean_df = df[~is_error].copy()

# 4. GENERATE CUSTOMERS
clean_df['Customer ID'] = clean_df['Customer ID'].fillna('Guest')
clean_df['CustomerId'] = clean_df['Customer ID'].apply(lambda x: 'Guest' if x == 'Guest' else str(int(float(x))))
cust_group = clean_df.groupby('CustomerId').first().reset_index()

def make_customer(row):
    cid = row['CustomerId']
    if cid == 'Guest': return pd.Series([cid, 'Guest Customer', 'guest@store.com', '000-000-0000', 'Guest Address', row['Country']])
    return pd.Series([cid, f"Customer {cid}", f"user{cid}@example.com", f"081-{cid[-9:]}", f"Address {cid}, {row['Country']}", row['Country']])

customer_df = pd.DataFrame()
customer_df[['CustomerId', 'CustomerName', 'CustomerEmail', 'CustomerPhone', 'CustomerAddress', 'CountryName']] = cust_group.apply(make_customer, axis=1)

countries = customer_df['CountryName'].unique()
country_map = {name: i+1 for i, name in enumerate(countries)}
customer_df['CountryId'] = customer_df['CountryName'].map(country_map)

# 5. GENERATE PRODUCTS (APPLYING STOCK LOGIC)
product_group = clean_df.groupby('StockCode').agg({'Description': 'last', 'Price': 'last'}).reset_index()
product_df = pd.DataFrame()
product_df['ProductId'] = product_group['StockCode']
product_df['CategoryId'] = 1
product_df['ProductName'] = product_group['Description'].fillna('Unknown').str.slice(0, 200)
product_df['ProductDescription'] = product_df['ProductName']
product_df['ProductPrice'] = product_group['Price']

# APPLY THE CALCULATED STOCK
# Base 100,000 + Net Change
product_df['NetChange'] = product_df['ProductId'].map(stock_impact_map).fillna(0)
product_df['ProductQuantity'] = 100000 + product_df['NetChange']
product_df['ProductQuantity'] = product_df['ProductQuantity'].astype(int)

# 6. EXPEDITION & SELLER
expedition_data = ['JNE', 'J&T', 'SiCepat', 'DHL', 'FedEx']
expedition_df = pd.DataFrame({'ExpeditionId': range(1, len(expedition_data) + 1), 'ExpeditionName': expedition_data})
seller_data = [('Tech Store', 'tech@store.com', '08123456789', 'Tech Street 1'), ('Fashion Hub', 'fashion@hub.com', '08129876543', 'Fashion Ave 2'), ('Home Goods', 'home@goods.com', '08133344455', 'Home Rd 3')]
seller_df = pd.DataFrame(seller_data, columns=['SellerName', 'SellerEmail', 'SellerPhone', 'SellerAddress'])
seller_df['SellerId'] = range(1, len(seller_df) + 1)
seller_df = seller_df[['SellerId', 'SellerName', 'SellerEmail', 'SellerPhone', 'SellerAddress']]

# 7. INVOICES & DETAILS
clean_df['LineTotal'] = clean_df['Quantity'] * clean_df['Price']

agg_df = clean_df.groupby(['Invoice', 'StockCode']).agg({
    'Quantity': 'sum', 'LineTotal': 'sum', 'InvoiceDate': 'first', 'CustomerId': 'first'
}).reset_index()
agg_df = agg_df[agg_df['Quantity'] != 0]

invoice_group = agg_df.groupby('Invoice').agg({'CustomerId': 'first', 'InvoiceDate': 'first'}).reset_index()
invoice_df = pd.DataFrame()
invoice_df['InvoiceId'] = invoice_group['Invoice']
invoice_df['CustomerId'] = invoice_group['CustomerId']
invoice_df['SellerId'] = np.random.choice(seller_df['SellerId'], size=len(invoice_df))
invoice_df['ExpeditionId'] = np.random.choice(expedition_df['ExpeditionId'], size=len(invoice_df))
invoice_df['InvoiceDate'] = invoice_group['InvoiceDate']
invoice_df['InvoiceStatus'] = invoice_df['InvoiceId'].astype(str).apply(lambda x: 'Returned' if x.startswith('C') else 'Paid')

details_df = pd.DataFrame()
details_df['InvoiceId'] = agg_df['Invoice']
details_df['ProductId'] = agg_df['StockCode']
details_df['Quantity'] = agg_df['Quantity']
details_df['TotalPrice'] = agg_df['LineTotal']
details_df = details_df[details_df['ProductId'].isin(set(product_df['ProductId']))]

# 8. EXPORT
quoting = csv.QUOTE_NONNUMERIC
category_df = pd.DataFrame({'CategoryId': [1], 'CategoryName': ['General']})
country_export = pd.DataFrame({'CountryId': list(country_map.values()), 'CountryName': list(country_map.keys())})

category_df.to_csv('import_category.csv', index=False, quoting=quoting)
country_export.to_csv('import_country.csv', index=False, quoting=quoting)
customer_df[['CustomerId', 'CustomerName', 'CustomerEmail', 'CustomerPhone', 'CustomerAddress', 'CountryId']].to_csv('import_customer.csv', index=False, quoting=quoting)
expedition_df.to_csv('import_expedition.csv', index=False, quoting=quoting)
seller_df.to_csv('import_seller.csv', index=False, quoting=quoting)
product_df.to_csv('import_product.csv', index=False, quoting=quoting, float_format='%.2f')
invoice_df.to_csv('import_invoice.csv', index=False, quoting=quoting)
details_df.to_csv('import_invoicedetails.csv', index=False, quoting=quoting, float_format='%.2f')

print("Files Created!")

Files Created!
